In [1]:
import numpy as np

# --------- Utility functions ----------
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2


# --------- Simple RNN Class ----------
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Weight initialization
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(output_size, hidden_size) * 0.01

        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    # --------- Forward Pass ----------
    def forward(self, inputs):
        h_prev = np.zeros((self.hidden_size, 1))
        self.hs = {}
        self.hs[-1] = h_prev
        self.inputs = inputs

        outputs = []

        for t in range(len(inputs)):
            x = inputs[t].reshape(-1, 1)
            h = tanh(self.Wxh @ x + self.Whh @ h_prev + self.bh)
            y = self.Why @ h + self.by

            self.hs[t] = h
            h_prev = h
            outputs.append(y)

        return outputs

    # --------- Backpropagation Through Time ----------
    def backward(self, doutputs, learning_rate=0.01):
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)

        dh_next = np.zeros((self.hidden_size, 1))

        for t in reversed(range(len(self.inputs))):
            dy = doutputs[t]
            dWhy += dy @ self.hs[t].T
            dby += dy

            dh = self.Why.T @ dy + dh_next
            dh_raw = dh * tanh_derivative(self.hs[t])

            dbh += dh_raw
            dWxh += dh_raw @ self.inputs[t].reshape(1, -1)
            dWhh += dh_raw @ self.hs[t - 1].T

            dh_next = self.Whh.T @ dh_raw

        # Gradient descent update
        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh -= learning_rate * dbh
        self.by -= learning_rate * dby


In [2]:
# Dummy sequence input
sequence = [
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 0, 1])
]

rnn = SimpleRNN(input_size=3, hidden_size=5, output_size=2)

# Forward pass
outputs = rnn.forward(sequence)

# Fake gradient
doutputs = [np.ones((2, 1)) for _ in outputs]

# Backward pass
rnn.backward(doutputs)
